In [185]:
import pandas as pd
import os

from formatacao_sumulas import FootballDataProcessor
from algorithm import FootballMatchRating
import numpy as np
import warnings
warnings.filterwarnings("ignore")


id_players = pd.read_excel('id_players_all.xlsx', index_col=0)


data_folder = '../Data/'


In [186]:
fix_id_players = pd.read_excel('ratings_2014_2023.xlsx')

In [187]:
fix_id_players = fix_id_players[['player', 'rating']]

In [188]:
# import pandas as pd
# import numpy as np
# from scipy.stats import skellam
# from scipy.optimize import fsolve

# def calculate_team_rating(sub_df):
#     """Calculate the weighted average rating of players on the field during a subgame."""
#     return sub_df['rating'].mean()

# def calculate_skellam_parameters(rating_home, rating_away, p_alvo=0.75):
#     """Calculate the parameters (mu1, mu2) for the Skellam distribution based on team ratings."""
#     soma = 3  # Define a fixed sum of expected goals (adjustable if needed)
#     p_alvo = 1/(1+10**((rating_away-rating_home)/400))

#     def equation(mu2, p_alvo):
#         mu1 = soma - mu2
#         p = 1 - skellam.cdf(0, mu1, mu2) + skellam.pmf(0, mu1, mu2) / 2
#         return p - p_alvo

#     mu2_inicial = 0.9  # Initial guess for mu2
#     mu2 = fsolve(equation, mu2_inicial, args=(p_alvo))[0]
#     mu1 = soma - mu2
#     return mu1, mu2

# def simulate_goals(E_home, E_away, duration_subgame):
#     """Simulate goals for home and away teams using Skellam trials based on expected scores."""
#     goals_home = np.random.poisson(E_home * duration_subgame/90, 1) 
#     goals_away = np.random.poisson(E_away * duration_subgame/90, 1) 

#     return goals_home, goals_away

# def process_game(df):
#     """Process an entire game to calculate goals for and against for each player based on subgames."""
    
#     # Step 1: Identify subgames based on substitutions
#     home_df = df[df['status'] == 'Home']
#     away_df = df[df['status'] == 'Away']

#     # Create a list of all substitution times for each team (including the start and end of the match)
#     sub_times_home = sorted(set(home_df['Minute Entered'].tolist() + home_df['Minute Exited'].tolist() + [0, 90]))
#     sub_times_away = sorted(set(away_df['Minute Entered'].tolist() + away_df['Minute Exited'].tolist() + [0, 90]))
    
#     # We will look at every interval (subgame) from one substitution to the next for both teams
#     subgame_intervals = sorted(set(sub_times_home + sub_times_away))

#     # Initialize columns for tracking goals for and against for each player.
#     df['Goals For'] = 0
#     df['Goals Against'] = 0

#     # Loop over each subgame interval
#     for i in range(1, len(subgame_intervals)):
#         start_minute = subgame_intervals[i - 1]
#         end_minute = subgame_intervals[i]

#         # Calculate duration of the subgame (in minutes)
#         duration_subgame = end_minute - start_minute

#         # Filter players who were on the field during this interval for both teams
#         home_on_field = home_df[(home_df['Minute Entered'] <= start_minute) & (home_df['Minute Exited'] >= end_minute)]
#         away_on_field = away_df[(away_df['Minute Entered'] <= start_minute) & (away_df['Minute Exited'] >= end_minute)]

#         # Calculate team ratings for this subgame
#         home_rating_subgame = calculate_team_rating(home_on_field)
#         away_rating_subgame = calculate_team_rating(away_on_field)

#         # Calculate Skellam parameters for this subgame
#         mu1, mu2 = calculate_skellam_parameters(home_rating_subgame, away_rating_subgame)

#         # Simulate goals for this subgame, adjusted by the duration of the subgame
#         goals_home_subgame, goals_away_subgame = simulate_goals(mu1, mu2, duration_subgame)

#         # Step 3: Assign goals to players based on their presence in the subgame
#         # Update 'Goals For' and 'Goals Against' for players in Home team
#         df.loc[df['player'].isin(home_on_field['player']), 'Goals For'] += goals_home_subgame
#         df.loc[df['player'].isin(home_on_field['player']), 'Goals Against'] += goals_away_subgame

#         # Update 'Goals For' and 'Goals Against' for players in Away team
#         df.loc[df['player'].isin(away_on_field['player']), 'Goals For'] += goals_away_subgame
#         df.loc[df['player'].isin(away_on_field['player']), 'Goals Against'] += goals_home_subgame

#     return df


In [189]:
import pandas as pd
import numpy as np
from scipy.stats import skellam
from scipy.optimize import fsolve

def calculate_team_rating(sub_df):
    """Calculate the weighted average rating of players on the field during a subgame."""
    return sub_df['rating'].mean()

def calculate_skellam_parameters(rating_home, rating_away, p_alvo=0.75):
    """Calculate the parameters (mu1, mu2) for the Skellam distribution based on team ratings."""
    soma = 3  # Define a fixed sum of expected goals (adjustable if needed)
    p_alvo = 1/(1+10**((rating_away-rating_home)/400))

    def equation(mu2, p_alvo):
        mu1 = soma - mu2
        p = 1 - skellam.cdf(0, mu1, mu2) + skellam.pmf(0, mu1, mu2) / 2
        return p - p_alvo

    mu2_inicial = 0.9  # Initial guess for mu2
    mu2 = fsolve(equation, mu2_inicial, args=(p_alvo))[0]
    mu1 = soma - mu2
    return mu1, mu2

def simulate_goals(E_home, E_away, duration_subgame):
    """Simulate goals for home and away teams using Skellam trials based on expected scores."""
    goals_home = np.random.poisson(E_home * duration_subgame/90, 1) 
    goals_away = np.random.poisson(E_away * duration_subgame/90, 1) 

    return goals_home, goals_away

def process_game(df):
    """Process an entire game to calculate goals for and against for each player based on subgames."""
    
    # Step 1: Identify subgames based on substitutions
    home_df = df[df['status'] == 'Home']
    away_df = df[df['status'] == 'Away']
    
    # Step 2: Calculate team ratings for the entire game (no subgame division)
    home_rating_game = calculate_team_rating(home_df)
    away_rating_game = calculate_team_rating(away_df)   
    mu1, mu2 = calculate_skellam_parameters(home_rating_game, away_rating_game) 

    # Create a list of all substitution times for each team (including the start and end of the match)
    sub_times_home = sorted(set(home_df['Minute Entered'].tolist() + home_df['Minute Exited'].tolist() + [0, 90]))
    sub_times_away = sorted(set(away_df['Minute Entered'].tolist() + away_df['Minute Exited'].tolist() + [0, 90]))
    
    # We will look at every interval (subgame) from one substitution to the next for both teams
    subgame_intervals = sorted(set(sub_times_home + sub_times_away))

    # Initialize columns for tracking goals for and against for each player.
    df['Goals For'] = 0
    df['Goals Against'] = 0

    # Loop over each subgame interval
    for i in range(1, len(subgame_intervals)):
        start_minute = subgame_intervals[i - 1]
        end_minute = subgame_intervals[i]

        # Calculate duration of the subgame (in minutes)
        duration_subgame = end_minute - start_minute

        # Filter players who were on the field during this interval for both teams
        home_on_field = home_df[(home_df['Minute Entered'] <= start_minute) & (home_df['Minute Exited'] >= end_minute)]
        away_on_field = away_df[(away_df['Minute Entered'] <= start_minute) & (away_df['Minute Exited'] >= end_minute)]

        # Simulate goals for this subgame, adjusted by the duration of the subgame
        goals_home_subgame, goals_away_subgame = simulate_goals(mu1, mu2, duration_subgame)

        # Step 3: Assign goals to players based on their presence in the subgame
        # Update 'Goals For' and 'Goals Against' for players in Home team
        df.loc[df['player'].isin(home_on_field['player']), 'Goals For'] += goals_home_subgame
        df.loc[df['player'].isin(home_on_field['player']), 'Goals Against'] += goals_away_subgame

        # Update 'Goals For' and 'Goals Against' for players in Away team
        df.loc[df['player'].isin(away_on_field['player']), 'Goals For'] += goals_away_subgame
        df.loc[df['player'].isin(away_on_field['player']), 'Goals Against'] += goals_home_subgame

    return df


In [190]:
# years = range(2013, 2015)
# campeonatos = ['Serie_A']

# for year in years:
#     for campeonato in campeonatos:
#         print(year, campeonato)
#         df_jogos_2014 = pd.read_json(f'../Data/{campeonato}_{year}_games.json')
#         df_jogos_2014_ = df_jogos_2014.T
#         df_jogos_2014_['id_game'] = range(1, len(df_jogos_2014_) + 1)

#         i=0
#         for index, row in df_jogos_2014_.iterrows():
#             if index-1 < len(df_jogos_2014_):
#                 processor = FootballDataProcessor(df_jogos_2014_, index-1)
#                 final_df = processor.process(data_folder, campeonato, year)
#                 final_df.rename(columns={'player_id': 'player'}, inplace=True)
#                 final_df['id_game'] = i+1
#                 final_df['player'] = final_df['player'].astype('string')
#                 fix_id_players['player'] = fix_id_players['player'].astype('string')
#                 final_df = pd.merge(final_df, fix_id_players, on='player', how='left')
#                 final_df = process_game(final_df)
                
#                 final_df = final_df.drop(columns=['rating'])
            
                
#                 id_players['player'] = id_players['player'].astype('string')

#                 final_df_ = pd.merge(final_df, id_players, on='player', how='left')
                


#                 team_1_name = df_jogos_2014_.iloc[index-1, 0]
#                 team_2_name = df_jogos_2014_.iloc[index-1, 1]

#                 # Criar uma instância de FootballMatchRating
#                 rating_calculator = FootballMatchRating(final_df_, team_1_name, team_2_name)

#                 updated_match_data = rating_calculator.update_ratings()

#                 teams_over_9_players = updated_match_data[updated_match_data['Minutes Played'] == 90] \
#                                         .groupby('team').filter(lambda x: len(x) > 10)

#                 updated_match_data['player'] = updated_match_data['player'].astype('string')

#                 updated_match_data.rename(columns={'player_name_x': 'player_name'}, inplace=True)
#                 updated_match_data.rename(columns={'time_jogador_x': 'time_jogador'}, inplace=True)

#                 updated_values = id_players[['player']].merge(
#                     updated_match_data[['player', 'rating', 'games_played', 'player_name', 'time_jogador']],
#                     on='player',
#                     how='left'
#                 )

#                 # Atualizar apenas as linhas corespondentes em id_players

#                 id_players.update(updated_values)
#             else:
#                 print(f"Índice {index-1} fora dos limites para o ano {year}")
#                 continue


# id_players


In [191]:
years = range(2014, 2024)
campeonatos = ['Serie_A']

for year in years:
    for campeonato in campeonatos:
        print(year, campeonato)
        df_jogos_2014 = pd.read_json(f'../Data/{campeonato}_{year}_games.json')
        df_jogos_2014_ = df_jogos_2014.T
        df_jogos_2014_['id_game'] = range(1, len(df_jogos_2014_) + 1)

        i=0
        for index, row in df_jogos_2014_.iterrows():
            if index-1 < len(df_jogos_2014_):
                processor = FootballDataProcessor(df_jogos_2014_, index-1)
                final_df = processor.process(data_folder, campeonato, year)
                
                # se o final_df for None ou vazio, pular para o próximo loop
                if final_df is None or final_df.empty:
                    continue
                
                if final_df[final_df['status'] == 'Home'].empty or final_df[final_df['status'] == 'Away'].empty:
                    continue
                
                if len(final_df) < 22:
                    continue
                
                final_df.rename(columns={'player_id': 'player'}, inplace=True)
                final_df['id_game'] = i+1
                final_df['player'] = final_df['player'].astype('string')
                fix_id_players['player'] = fix_id_players['player'].astype('string')
                final_df = pd.merge(final_df, fix_id_players, on='player', how='left')
                final_df = process_game(final_df)
                
                final_df = final_df.drop(columns=['rating'])
                id_players['player'] = id_players['player'].astype('string')

                final_df_ = pd.merge(final_df, id_players, on='player', how='left')
                
                team_1_name = df_jogos_2014_.iloc[index-1, 0]
                team_2_name = df_jogos_2014_.iloc[index-1, 1]

                # Criar uma instância de FootballMatchRating
                rating_calculator = FootballMatchRating(final_df_, team_1_name, team_2_name)

                updated_match_data = rating_calculator.update_ratings()

                teams_over_9_players = updated_match_data[updated_match_data['Minutes Played'] == 90] \
                                        .groupby('team').filter(lambda x: len(x) > 10)

                updated_match_data['player'] = updated_match_data['player'].astype('string')

                updated_match_data.rename(columns={'player_name_x': 'player_name'}, inplace=True)
                updated_match_data.rename(columns={'time_jogador_x': 'time_jogador'}, inplace=True)

                updated_values = id_players[['player']].merge(
                    updated_match_data[['player', 'rating', 'games_played', 'player_name', 'time_jogador']],
                    on='player',
                    how='left'
                )

                # Atualizar apenas as linhas corespondentes em id_players

                id_players.update(updated_values)
            else:
                print(f"Índice {index-1} fora dos limites para o ano {year}")
                continue


id_players


2014 Serie_A
2015 Serie_A
2016 Serie_A
Índice 379 fora dos limites para o ano 2016
2017 Serie_A
2018 Serie_A
2019 Serie_A
2020 Serie_A
2021 Serie_A
2022 Serie_A
2023 Serie_A


,player,rating,age,games_played,player_name,time_jogador
0,328974,1504.585683,25,1,3Talysson L ...,Coritiba / PR
1,407969,1500.000000,25,0,NaN,NaN
2,595214,1500.000000,25,0,NaN,NaN
3,612866,1500.000000,25,0,NaN,NaN
4,138485,1500.000000,25,0,NaN,NaN
...,...,...,...,...,...,...
14892,441035,1500.000000,25,0,NaN,NaN
14893,184992,1500.000000,25,0,NaN,NaN
14894,295793,1500.000000,25,0,NaN,NaN
14895,465944,1492.399595,25,5,90Negueba Gabriel dos Santos F ...,Ceará / CE


In [214]:
# unir os dados por player e calcular o coeficiente de correlação de spearman
import numpy as np
from scipy.stats import spearmanr
list_n = [10, 100, 500]
x = id_players.sort_values(by='rating', ascending=False)
for i in list_n:
    acertos = x.head(i)['player'].isin(fix_id_players.sort_values(by='rating', ascending=False).head(i)['player']).value_counts().get(True, 0)
    proporcao_acertos = acertos / i
    print(f"Proporção de acertos para {i} primeiros jogadores: {proporcao_acertos}")
    print('')


df_ = pd.merge(fix_id_players.sort_values(by='rating', ascending=False), id_players.sort_values(by='rating', ascending=False), on='player', how='left')
df_ = df_.dropna()
coeficiente, p_valor = spearmanr(df_['rating_x'], df_['rating_y'])

print(f"Coeficiente de Correlação de Spearman total: {coeficiente}")
print(f"Valor-p: {p_valor}")

Proporção de acertos para 10 primeiros jogadores: 0.2

Proporção de acertos para 100 primeiros jogadores: 0.33

Proporção de acertos para 500 primeiros jogadores: 0.45

Coeficiente de Correlação de Spearman total: 0.3446330176500481
Valor-p: 8.8382470751599e-81
